# Traduzione

Una delle forze motrici che ha permesso lo sviluppo della civiltà umana è la capacità di comunicare reciprocamente. Nella maggior parte delle attività umane, la comunicazione è fondamentale.

![Un robot multilingue](./images/translation.jpg)

L'intelligenza artificiale (IA) può aiutare a semplificare la comunicazione traducendo il testo o il parlato tra le lingue, aiutando a rimuovere le barriere alla comunicazione tra paesi e culture.

## Crea una risorsa di servizi cognitivi

In Azure, puoi usare i servizi cognitivi per tradurre tra più lingue.

Se non ne hai già una, procedi come segue per creare una risorsa di **Servizi Cognitivi** nella tua sottoscrizione di Azure:

> **Nota**: Se disponi già di una risorsa di Servizi Cognitivi, basta aprire la sua pagina **Avvio rapido** nel portale di Azure e copiare la sua chiave e l'endpoint nella cella seguente. Altrimenti, procedi come segue per crearne una.

1. In un'altra scheda del browser, apri il portale di Azure all'indirizzo https://portal.azure.com, accedendo con il tuo account Microsoft.
2. Fai clic sul pulsante **&#65291;Crea una risorsa**, cerca *Servizi cognitivi* e crea una risorsa di **Servizi cognitivi** con le impostazioni seguenti:
    - **Sottoscrizione**: *La tua sottoscrizione di Azure*.
    - **Gruppo di risorse**: *Seleziona o crea un gruppo di risorse con un nome univoco*.
    - **Area geografica**: *Scegli una qualsiasi area disponibile*:
    - **Nome**: *Immetti un nome univoco*.
    - **Piano tariffario**: S0
    - **Confermo di aver letto e compreso gli avvisi**: Selezionato.
3. Attendi il completamento della distribuzione. Vai quindi alla tua risorsa di servizi cognitivi e, nella pagina **Panoramica**, fai clic sul link per gestire le chiavi per il servizio. Avrai bisogno dell'endpoint e delle chiavi per connetterti alla tua risorsa di servizi cognitivi dalle applicazioni client.

### Ottieni la chiave e la posizione per la tua risorsa di Servizi cognitivi

Per usare la risorsa di servizi cognitivi, le applicazioni client hanno bisogno della chiave di autenticazione e della posizione:

1. Nel portale di Azure, nella pagina **Chiavi ed endpoint** per la tua risorsa di servizio cognitivo, copia la **Key1** per la tua risorsa e incollala nel codice sottostante, sostituendo **YOUR_COG_KEY**.
2. Copia la **Posizione** per la tua risorsa e incollala nel codice sottostante, sostituendo **YOUR_COG_LOCATION**.
>**Nota**: Rimani nella pagina **Chiavi ed endpoint** e copia la **Posizione** da questa pagina (esempio: _westus_). Non aggiungere spazi tra le parole per il campo Posizione. 
3. Esegui il codice seguente facendo clic sul pulsante **Esegui cella** (&#9655;) a sinistra della cella.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_location = 'YOUR_COG_LOCATION'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

## Traduzione del testo

Come si evince dal nome, il servizio **Traduzione testuale** permette di tradurre il testo da una lingua all'altra.

Non esiste un SDK Python per questo servizio, ma è possibile utilizzare la sua interfaccia REST per inviare richieste ad un endpoint su HTTP: una procedura relativamente semplice da eseguire in Python utilizzando la libreria **requests**. Le informazioni sul testo da tradurre e il testo tradotto risultante sono scambiati in formato JSON.

Esegui la seguente cella per creare una funzione che svolga questa operazione, e poi testala con una semplice traduzione dall'inglese al francese.

In [ ]:
# Create a function that makes a REST request to the Text Translation service
def translate_text(cog_location, cog_key, text, to_lang='fr', from_lang='en'):
    import requests, uuid, json

    # Create the URL for the Text Translator service REST request
    path = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'
    params = '&from={}&to={}'.format(from_lang, to_lang)
    constructed_url = path + params

    # Prepare the request headers with Cognitive Services resource key and region
    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region':cog_location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Add the text to be translated to the body
    body = [{
        'text': text
    }]

    # Get the translation
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]["text"]


# Test the function
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='fr', from_lang='en')
print('{} -> {}'.format(text_to_translate,translation))

Il servizio dovrebbe aver tradotto il testo inglese "Hello" al francese "Bonjour".

Tieni presente che le lingue sono specificate usando un sistema standard di abbreviazioni linguistiche, con *en* per l'inglese e *fr* per il francese. Puoi anche usare abbreviazioni che includono culture specifiche: è utile quando la stessa lingua è usata in diverse aree geografiche, spesso con grafie diverse. Ad esempio *en-US* indica l'inglese negli Stati Uniti, mentre *en-GB* indica l'inglese in Gran Bretagna.

Esegui la seguente cella per tradurre tra l'inglese britannico e l'italiano.

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='it-IT', from_lang='en-GB')
print('{} -> {}'.format(text_to_translate,translation))

Proviamo un'altra traduzione, questa volta dall'inglese americano al cinese.

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='zh-CN', from_lang='en-US')
print('{} -> {}'.format(text_to_translate,translation))

## Traduzione vocale

Puoi usare il servizio **Voce** per tradurre la lingua parlata.

Ora puoi eseguire la seguente cella per creare e testare una funzione che utilizza l'SDK Voce per tradurre la voce udibile.

In [ ]:
# Create a function to translate audio in one language to text in another
def translate_speech(cog_location, cog_key, audio_file=None, to_lang='fr-FR', from_lang='en-US'):
    from azure.cognitiveservices.speech import SpeechConfig, AudioConfig, ResultReason
    from azure.cognitiveservices.speech.translation import SpeechTranslationConfig, TranslationRecognizer

    # Configure the speech translation service
    translation_config = SpeechTranslationConfig(subscription=cog_key, region=cog_location)
    translation_config.speech_recognition_language = from_lang
    translation_config.add_target_language(to_lang)

    # Configure audio input
    if audio_file is None:
        audio_config = AudioConfig() # Use default input (microphone)
    else:
        audio_config = AudioConfig(filename=audio_file) # Use file input

    # Create a translation recognizer and use it to translate speech input
    recognizer = TranslationRecognizer(translation_config, audio_config)
    result = recognizer.recognize_once()

    # Did we get it?
    translation = ''
    speech_text = ''
    if result.reason == ResultReason.TranslatedSpeech:
        speech_text = result.text
        translation =  result.translations[to_lang]
    elif result.reason == ResultReason.RecognizedSpeech:
        speech_text = result.text
        translation =  'Unable to translate speech'
    else:
        translation = 'Unknown'
        speech_text = 'Unknown'

    # rturn the translation
    return speech_text, translation
    

# Test the function
import os

file_name = 'english.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='es', from_lang='en-US')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
print(result)

Tieni presente che la lingua "a" deve essere identificata con un codice lingua di 2 caratteri (ad esempio *en*), mentre la lingua "da" deve includere l'indicatore di cultura (ad esempio *en-US*).

Proviamo a tradurre dal francese all'inglese.

In [ ]:
import os

file_name = 'french.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='en', from_lang='fr-FR')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
print(result)

## Scopri di più

Puoi sapere di più su [Traduzione testuale](https://docs.microsoft.com/azure/cognitive-services/translator/) e sulla [traduzione con il servizio Voce](https://docs.microsoft.com/azure/cognitive-services/speech-service/index-speech-translation) nella documentazione del servizio.